In [1]:
# initial setup
try:
    # settings colab:
    import google.colab
        
except ModuleNotFoundError:    
    # settings local:
    %run "common/0_notebooks_base_setup.py"

default checking
Running command `conda list`... ok
jupyterlab=1.2.6 already installed
pandas=1.0.3 already installed
matplotlib=2.2.2 already installed
bokeh=2.0.0 already installed
seaborn=0.10.0 already installed
ipywidgets=7.5.1 already installed
pytest=5.3.4 already installed
chardet=3.0.4 already installed
psutil=5.7.0 already installed
scipy=1.4.1 already installed
statsmodels=0.11.0 already installed
scikit-learn=0.22.1 already installed
xlrd=1.2.0 already installed
plotly=4.6.0 already installed
chart-studio=1.1.0 already installed
plotly-orca=1.3.1 already installed
Running command `conda install --yes unicodedata2`... ok
Solving environment: ...working... done

# All requested packages already installed.


Importando librerías ...
Las librerías has sido importadas con éxito


In [2]:
data = pd.read_csv("properati.csv")

In [3]:
data_clean = pd.read_csv("CleanCSV/Clean01_TipoOperacion.csv")

<hr id="Dummies">
<h2 style="color: lightblue">VARIABLES DUMMIE Y SUPERFICIE</h2>

### Generamos una lista de palabras claves obtenidas de la columna 'properati_url' ordenadas por cantidad de repeticiones

In [4]:
pattern = r'http:\/\/www\.properati.com.ar\/\w{6}([\w*-]*)'
pattern_regex = re.compile(pattern)
result_match = data['properati_url'].apply(lambda x: pattern_regex.match(x.lower().replace('-','_')))
result =  result_match.apply(lambda x: x.group(1).split('_'))

list_words = []

def generate_words_series(arr):
    for x in arr:
        list_words.append(str(x))
    return

result.apply(generate_words_series)

serie = pd.Series(list_words)
key_words = serie.value_counts()

In [5]:
display(key_words.head(40))

venta            121119
departamento      71019
garage            56620
lavadero          44034
balcon            42288
propiedades       41507
casa              40537
parrilla          33381
piscina           32148
luminoso          31865
suite             27102
placard           26145
terraza           25101
toilette          22841
subte             21984
linea             21824
patio             21169
vestidor          18011
jardin            17173
quincho           12850
aire              12767
acondicionado     12763
inmobiliaria      12184
sum               11665
dependencias      11316
max               11260
re                11258
cordoba           11195
amenities         10607
baulera           10512
estrenar          10383
vista             10309
de                 9768
san                9556
gimnasio           9363
lujoso             9251
plata              9215
del                9180
rosario            8607
villa              8473
dtype: int64

### a partir de estas palabras claves identificamos las que nos pueden servir para:

### - generar variables dummy:
* garage
* lavadero
* balcon  
* casa
* parrilla
* piscina
* terraza
* patio
* jardin
* quincho
* aire acondicionado
* amenities
* estrenar
* gimnasio

opcionales..
* baulera
* suite
* subte - linea
* vestidor

### - completar datos faltantes
* venta
* departamento



## Ahora hacemos lo mismo con la columna 'descripcion'

In [6]:
pattern_desc = r'(\w\w\w\w\w*)'
pattern_desc_regex = re.compile(pattern_desc, flags=re.IGNORECASE )
result_match = data['description'].apply(lambda x: re.finditer(pattern_desc_regex, str(x).lower()))

list_words_desc = []

def generate_words_series_desc(matches):
    for matchNum, match in enumerate(matches, start=1):
        for groupNum in range(0, len(match.groups())):
            groupNum = groupNum + 1
            
            cadena = unicodedata.normalize('NFKD', match.group(groupNum)).encode('ASCII', 'ignore') 
            
            list_words_desc.append(cadena)
    return

result =  result_match.apply(generate_words_series_desc)

serie_desc = pd.Series(list_words_desc)
key_words_desc = serie_desc.value_counts()

In [7]:
display(key_words_desc.head(40))

b'cocina'          110088
b'bano'             98520
b'comedor'          96205
b'para'             85190
b'living'           70516
b'dormitorios'      67240
b'ambientes'        56062
b'dormitorio'       53025
b'piso'             50769
b'completo'         49287
b'departamento'     48517
b'balcon'           47086
b'pisos'            45367
b'planta'           44549
b'frente'           43095
b'casa'             39426
b'lavadero'         39128
b'excelente'        37242
b'amplio'           36923
b'edificio'         36296
b'venta'            35989
b'parrilla'         33136
b'cochera'          31810
b'cuenta'           31541
b'placard'          31247
b'suite'            31146
b'sobre'            28899
b'terraza'          28332
b'patio'            25833
b'vista'            24966
b'mesada'           24908
b'gran'             24171
b'propiedad'        23457
b'baja'             23351
b'ubicado'          22884
b'barrio'           22826
b'esta'             22535
b'zona'             22516
b'servicio' 

### Palabras para generar variables dummy
* balcon
* frente
* parrilla
* cochera
* terraza


Opcionales:
suite

### Palabras para completar datos faltantes
* dormitorio/s
* ambientes
* piso - planta
* departamento - casa - edificio
* venta
* ubicado - barrio

### Creamos una funcion que nos permita:

* Observar el contexto de una palabra clave seleccionada dentro de la descripcion
* Obtener un set de indices de filas que en su descripcion contienen esa palabra
* Determinar cuantas filas queremos recorrer

In [8]:
def generate_context_df(word, column, rows=5000, has_context=True):
    global data_index
    if has_context is True:
        pattern_context = r'(?P<context>(\w*\W?\s\W?){0,4}' + word + '(\w*\W?\s\W?){0,4})'
    else:    
        pattern_context = word

    pattern_context_regex = re.compile(pattern_context, flags=re.IGNORECASE )
    result_match = data[column].apply(lambda x: re.finditer(pattern_context_regex, str(x).lower()))

    list_words_desc_context = []
    list_words_desc_index = []
    data_index = 0
    df_context = pd.DataFrame()
        
    def generate_context_df(matches):
        global data_index
        for matchNum, match in enumerate(matches, start=1):  
            if has_context is True:
                context = match.group('context')     
                list_words_desc_context.append(context)
            list_words_desc_index.append(data_index)
        data_index += 1
        return
    
    if rows == 0:
        result = result_match.apply(generate_context_df)
    else:
        result = result_match[0:rows].apply(generate_context_df)
        
    if has_context is True:
        df_context['context'] = list_words_desc_context
        
    df_context['index'] = list_words_desc_index
    print('cantidad de matches: ', df_context.shape[0], ' en ', rows if rows != 0 else 'TODOS los' , ' registros buscados')
    return df_context

### Comprobamos su funcionamiento solicitando 5000 registros de la potencial variable dummy 'garage|cochera'

In [9]:
generate_context_df("garage|cochera", "description", 5000, True).head(10)

cantidad de matches:  2726  en  5000  registros buscados


,context,index
0,cochera semicubierta.,1
1,cochera,10
2,"cochera y jardín propio,",15
3,cochera fija subterranea y,17
4,cochera. en cumplimiento de,23
5,cochera. entrega marzo,24
6,cochera,24
7,cochera. entrega marzo,25
8,cochera,25
9,cochera. entrega marzo,28


### Creamos una funcion que nos permite generar variables dummies a partir de palabras clave

In [10]:
def create_dummy(data, data_dummy, column_name):
    data_dummy.drop_duplicates(inplace=True)
    col_dummy_name = column_name + '_dummy'
    data_dummy[col_dummy_name] = True
    data_clean_dummy = pd.merge(data, data_dummy, how='left', left_on='Unnamed: 0', right_on='index')
    del data_clean_dummy['index']
    data_clean_dummy[col_dummy_name].fillna(False, inplace=True)
    print('Col generada: ' + column_name + '_dummy')
    return data_clean_dummy

### Creamos una lista de palabras claves que recorremos utilizando la funcion mencionada anteriormente

como resultado vamos a haber agregado a nuestro DataFrame una columna por cada variable dummy

In [11]:
list_columnas_dummy = ['garage|cochera', 'lavadero', 'balcon', 'parrilla', 'piscina', 'terraza', 'patio', 'jardin', 'quincho', 'aire acondicionado', 'amenities', 'estrenar', 'gimnasio', 'frente', 'subte', 'baulera' ]
for columna in list_columnas_dummy:
    data_dummy = generate_context_df(columna, "description", 0, False)
    data_clean =create_dummy(data_clean, data_dummy, columna)

cantidad de matches:  63489  en  TODOS los  registros buscados
Col generada: garage|cochera_dummy
cantidad de matches:  40637  en  TODOS los  registros buscados
Col generada: lavadero_dummy
cantidad de matches:  19817  en  TODOS los  registros buscados
Col generada: balcon_dummy
cantidad de matches:  38015  en  TODOS los  registros buscados
Col generada: parrilla_dummy
cantidad de matches:  18059  en  TODOS los  registros buscados
Col generada: piscina_dummy
cantidad de matches:  35678  en  TODOS los  registros buscados
Col generada: terraza_dummy
cantidad de matches:  27922  en  TODOS los  registros buscados
Col generada: patio_dummy
cantidad de matches:  9955  en  TODOS los  registros buscados
Col generada: jardin_dummy
cantidad de matches:  14636  en  TODOS los  registros buscados
Col generada: quincho_dummy
cantidad de matches:  12429  en  TODOS los  registros buscados
Col generada: aire acondicionado_dummy
cantidad de matches:  12381  en  TODOS los  registros buscados
Col generada

In [12]:
data_clean[data_clean.columns[-16:]].head(3)

,garage|cochera_dummy,lavadero_dummy,balcon_dummy,parrilla_dummy,piscina_dummy,terraza_dummy,patio_dummy,jardin_dummy,quincho_dummy,aire acondicionado_dummy,amenities_dummy,estrenar_dummy,gimnasio_dummy,frente_dummy,subte_dummy,baulera_dummy
0,False,True,False,False,False,False,True,False,False,False,False,False,False,False,False,False
1,True,True,False,False,False,False,False,False,False,False,False,True,False,True,False,False
2,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False


### Ahora vamos a trabajar sobre la columna 'descripcion' para:

* capturar los valores en m2 de las descripciones donde existan faltantes
* obervar el contexto de los valores capturados
* preservar el indice para una posterior imputación de datos
* crear una lista de palabras claves a buscar ordenadas por cantidad de repeticiones

In [13]:
#tomamos solo las filas con faltantes en los campos de m2
#normalizamos las cadenas de texto para remover caracteres especiales
mask_m2_any_null = data.surface_total_in_m2.isnull() | data.surface_covered_in_m2.isnull()
data_m2_null = data.loc[mask_m2_any_null, 'description'].apply(lambda x: unicodedata.normalize('NFKD', str(x)).encode('utf-8', 'ignore').lower())

#generamos un patron que captura el valor de los m2 y el contexto en el que fue encontrado
pattern_m2 = r'(?P<before>(\w*\W?\s\W?){0,4})\s?(?P<value>\d{1,5})\s*(m2|metros)\s?(?P<after>(\W?\w*\W?\s?){0,4})'

pattern_m2_regex = re.compile(pattern_m2, flags=re.IGNORECASE )
result_match = data_m2_null.apply(lambda x: re.finditer(pattern_m2_regex, str(x)))

df_result_match = result_match.to_frame().copy()
df_result_match['indice'] = result_match.to_frame().index
df_result_match

list_words_desc_after = []
list_words_desc_before = []
list_words_desc_values = []
list_words_desc_index = []

df_m2 = pd.DataFrame(columns=('before', 'm2', 'after', 'index'))

#recorremos los resultados
for index, row in df_result_match.iterrows():
    matches = row['description']
    indice = row['indice']
    for match in matches:  
        before = match.group('before')
        after =  match.group('after')
        value = match.group('value').strip()    

        list_words_desc_before.append(before)
        list_words_desc_values.append(value)
        list_words_desc_index.append(indice)
        list_words_desc_after.append(after)

df_m2['m2'] = list_words_desc_values
df_m2['before'] = list_words_desc_before
df_m2['index'] = list_words_desc_index
df_m2['after'] = list_words_desc_after
display(df_m2.head(20))
print('cantidad de matches: ', df_m2.size)

,before,m2,after,index
0,y servicios. lote de,1514,.,10
1,,23,contado u$d250.000haus,15
2,x83o de,50,"cuadrados,con terraza y",65
3,balcon. super luminoso tiene,38,aprox. cuenta con ban\,66
4,monoambientes de,22,+ 5.30m21 dormitorio,101
5,al frente de,36,+ 5.30m21 dormitorio,101
6,,50,+ 2.30m22 dormitorios,101
7,,50,+ 2.70m2entrega enero,101
8,monoambientes de,22,+ 5.30m21 dormitorio,102
9,al frente de,36,+ 5.30m21 dormitorio,102


cantidad de matches:  75476


### Generamos una lista de palabras claves tomadas del contexto del DataFrame anterior
### Estas palabras nos permitirán, luego, imputar datos faltantes en m2

In [14]:
keys_m2 = list_words_desc_before + list_words_desc_after
list_keys_words_m2 = []
for key in keys_m2:
    list_key_words = key.replace('.', '').replace(',', '').replace(':', '').split(' ')
    for word in list_key_words:
        if (word != '') & (len(word) > 2):
            list_keys_words_m2.append(word)
        
list_keys_words_m2

serie_keys_words_m2 = pd.Series(list_keys_words_m2)
serie_keys_words_m2_counts = serie_keys_words_m2.value_counts()
serie_keys_words_m2_counts.head(30)

con             3685
superficie      2197
cubiertos       1587
lote            1458
total           1192
cubierta        1191
sobre           1180
terreno         1156
una              956
sup              802
frente           760
casa             719
planta           712
x81n             704
aprox            688
ambientes        669
totales          600
dos              585
por              583
del              575
cuadrados        565
departamento     541
los              534
terraza          493
cocina           471
piso             470
cuenta           452
dormitorios      452
living           451
comedor          410
dtype: int64

### Elegimos de la lista las palabras mas relevantes
### Creamos a partir de ellas dos series con los cuales imputaremos valores en las columnas 'surface_total_in_m2' y 'surface_covered_in_m2'

In [15]:
#Unimos las columnas de contexto
df_m2['context'] = df_m2['before'] + " " + df_m2['after']

#buscamos los m2 totales
WORD = "total" #buscamos los m2 totales

key_word_mask_totales = df_m2['context'].apply(lambda x: True if WORD in x else False)
print('cantidad de veces que', WORD.upper(), 'fué encontrada: ', key_word_mask_totales.sum())

#df_m2_totales = df_m2.loc[key_word_mask,["m2","index"]]
df_m2_totales = df_m2.loc[key_word_mask_totales,["m2","index"]]

#buscamos los m2 cubiertos
list_words = ["sup cub", "cubiert"] 

def find_list_words_m2(list, row):
    existe = False
    for word in list:
        if word in row:
            existe = True
    return existe

key_word_mask_cubiertos = df_m2['context'].apply(lambda x : find_list_words_m2(list_words, x))
print('cantidad de veces que *sup cub* y *cubiert* fueron encontradas: ', key_word_mask_cubiertos.sum())

df_m2_cubiertos = df_m2.loc[key_word_mask_cubiertos,["m2","index", "context"]]

cantidad de veces que TOTAL fué encontrada:  1969
cantidad de veces que *sup cub* y *cubiert* fueron encontradas:  3641


In [16]:
m2_totales_a_imputar = df_m2_totales.groupby('index').max()
m2_totales_a_imputar

,m2
index,
175,88
491,122
538,2290
539,2290
869,84
...,...
120585,110
120633,56
120634,655


### Reemplazamos los valores obtenidos para 'm2 totales'

In [17]:
print("\n ..data_clean.loc[m2_totales_a_imputar.index, 'surface_total_in_m2']:")
display(data_clean.loc[m2_totales_a_imputar.index, 'surface_total_in_m2']) 
# ALGUNOS YA TIENE VALORES, NO LOS VAMOS A SOBREESCRIBIR.
mask_m2_total_imputar_null = data_clean.loc[m2_totales_a_imputar.index, 'surface_total_in_m2'].isnull()
mask_m2_total_imputar_null_final= mask_m2_total_imputar_null[mask_m2_total_imputar_null]   
# SOLO VAMOS A IMPUTAR DONDE NO TENGO NINGÚN VALOR
print("\n.. data_clean.loc[mask_m2_total_imputar_null_final.index, 'surface_total_in_m2']:")
data_clean.loc[mask_m2_total_imputar_null_final.index, 'surface_total_in_m2']


 ..data_clean.loc[m2_totales_a_imputar.index, 'surface_total_in_m2']:


index
175         NaN
491         NaN
538         NaN
539         NaN
869         NaN
          ...  
120585      NaN
120633      NaN
120634      NaN
120722    116.0
120929    287.0
Name: surface_total_in_m2, Length: 1827, dtype: float64


.. data_clean.loc[mask_m2_total_imputar_null_final.index, 'surface_total_in_m2']:


index
175      NaN
491      NaN
538      NaN
539      NaN
869      NaN
          ..
120184   NaN
120527   NaN
120585   NaN
120633   NaN
120634   NaN
Name: surface_total_in_m2, Length: 1549, dtype: float64

In [18]:
print('\n ..data_clean:')
display(data_clean.loc[mask_m2_total_imputar_null_final.index, 'surface_total_in_m2'])
data_clean.loc[mask_m2_total_imputar_null_final.index, 'surface_total_in_m2'] = \
                                                m2_totales_a_imputar.loc[mask_m2_total_imputar_null_final.index, 'm2']

print('\n Luego de imputarlos..data_clean:')
display(data_clean.loc[mask_m2_total_imputar_null_final.index, 'surface_total_in_m2'])

print('\n m2_totales_a_imputar:')
m2_totales_a_imputar.loc[mask_m2_total_imputar_null_final.index, 'm2']


 ..data_clean:


index
175      NaN
491      NaN
538      NaN
539      NaN
869      NaN
          ..
120184   NaN
120527   NaN
120585   NaN
120633   NaN
120634   NaN
Name: surface_total_in_m2, Length: 1549, dtype: float64


 Luego de imputarlos..data_clean:


index
175         88
491        122
538       2290
539       2290
869         84
          ... 
120184      95
120527      89
120585     110
120633      56
120634     655
Name: surface_total_in_m2, Length: 1549, dtype: object


 m2_totales_a_imputar:


index
175         88
491        122
538       2290
539       2290
869         84
          ... 
120184      95
120527      89
120585     110
120633      56
120634     655
Name: m2, Length: 1549, dtype: object

### Pequeña verificación de la asignación

In [19]:
print('\n ..data_clean:')
display(data_clean.loc[537:540, 'surface_total_in_m2'])
print('\n ..m2_totales_a_imputar:')
display(m2_totales_a_imputar.loc[537:540, 'm2'])


 ..data_clean:


537     NaN
538    2290
539    2290
540     NaN
Name: surface_total_in_m2, dtype: object


 ..m2_totales_a_imputar:


index
538    2290
539    2290
Name: m2, dtype: object

In [20]:
correccionSurfaceTotalInM2 = data_clean.surface_total_in_m2.apply(lambda x: str(x).lower().replace("m2", "").replace("superficie", "").strip())
correccionSurfaceTotalInM2Final = correccionSurfaceTotalInM2.apply(lambda x: x if x[0] != "." else x[1:]).astype(np.float64)
data_clean.loc[correccionSurfaceTotalInM2Final.index, "surface_total_in_m2"] = correccionSurfaceTotalInM2Final

In [23]:
property_type_dummies = pd.get_dummies(data_clean, columns=["property_type"], prefix="tipo_propiedad")
data_clean = pd.concat([data_clean, property_type_dummies], axis=1)

In [21]:
data_clean.drop(
    ['Unnamed: 0'], \
    axis=1, inplace=True)

In [24]:
data_clean.to_csv("CleanCSV/Clean02_DummiesYSuperficie.csv")